In [1]:
openai_key = "" ## Please change it to your key

In [2]:
## Comment this out if you run on colab
##!pip install openai 
##!pip install backoff

In [2]:
import os
import openai
import numpy as np
import pandas as pd
import time
import backoff

openai.api_key = openai_key

In [3]:
## We don't need to maintain message history for chat mode since 
## we will concat the prompts if history is true.
## Also no system message to make it comparable to text-davinci-003
## We test the snapshot so that the results are reproducible.
## The model are "text-davinci-003", "gpt-3.5-turbo", "gpt-4"
models = ["text-davinci-003", "gpt-3.5-turbo", "gpt-4"]

## Backofff for chat since both chatgpt and gpt-4 are always heavy used.
@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_time=6000)
def completions_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_time=6000)
def chat_completions_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

def gptQuery(prompt, model, temperature = 0, n=1, logprobs=1, echo = False, **kwargs):
  if model == models[0]:
    out=completions_with_backoff(model=model, 
                               prompt=prompt, 
                               logprobs=logprobs, 
                               temperature=temperature, max_tokens = 2048,
                               n = n, **kwargs)
    # out=openai.Completion.create(model=model, 
    #                            prompt=prompt, 
    #                            logprobs=logprobs, 
    #                            temperature=temperature, max_tokens = 2048,
    #                            n = n, **kwargs)
    if echo: print(out)
    return [response.text.strip() for response in out.choices ]
  if model == models[1] or model == models[2]:
    out = chat_completions_with_backoff(model=model,
                                     messages=[{"role":"user","content":prompt}], 
                                     temperature=temperature, max_tokens = 2048,
                                     n=n)
    # out = openai.ChatCompletion.create(model=model,
    #                                  messages=[{"role":"user","content":prompt}], 
    #                                  temperature=temperature, max_tokens = 2048,
    #                                  n=n, **kwargs)
    if echo: print(out) 
    return [response.message.content.strip() for response in out.choices ]

## Test that the openAIkey is working correctly
print(gptQuery(prompt = "Hello Test Test", model=models[0]))
print(gptQuery(prompt = "Hello Test Test", model=models[1]))
print(gptQuery(prompt = "Hello Test Test", model=models[2]))

["Hello! It's nice to meet you. How can I help you?"]
['Hello! How can I assist you today?']
['Hello! How can I help you today? If you have any questions or need assistance, feel free to ask.']


#Declaring Perrez Story

The stories below are from the Stress and Coping Process Questionnaire (SCPQ) from the Stress, coping, and health: A situation-behavior approach: Theroy, methods, and applications book by Perrez and Reicherts 1992.

Each story consists of three phases: Onset, Continuation, and Outcome.
The order is the same as the original study.

The story can be classifed into two type: Aversive and Lose & Failure. 

 - Aversive: 1, 3, 6, 8, 9, 10, 12, 14, 16.
 - Loss & Failure: 2, 4, 5, 7, 8, 11, 13 ,17, 18.

Aversive stories can be breakdown into two domains:
- Social:
  - Source: close, acquainted, strange persons
- Professional: 
  - Source: work activity, equal, superior persons

Loss and Failure can be breakdown into two domains:
- Social:
  - Sources: persons, activities/projects, objects
- Professional:
  - Sources: persons, activities/projects, objects

The story have two possible outcomes: Positive or Negative.

 - Positive: 1, 3, 4, 5, 8, 11, 14, 17, 18
 - Negative: 2, 6, 7, 9, 10, 12, 13, 15, 16

In [4]:
## Need to adjust for indexing starting at 0
AVERSIVE = "aversive"
LOSSFAILURE = "lossfailure"

aversive_stories = [1,3,6,8,9,10,12,14,16] 
lossfailure_stories = [2,4,5,7,8,11,13,17,18]

types = [ AVERSIVE if i in aversive_stories else LOSSFAILURE for i in range(1,19)]


pos = [1,3,5,8,11,14,17,18]
neg = [2,6,7,9,10,12,13,15,16]

## Aversive: Critism from the partner
story1 = ["You have forgotten to do something important for your partner. You become aware of it just at the moment when your partner asks about it. Your partner gets very angry and blames you.",
          "After a while, your partner’s attitude to you has hardly changed: he/she is still angry, and he still blames you for your mistake.",
          "At least your partner has apologized for having been so vehement. You work together to try and find a way to repair the damage."]

## Loss&Failure: Loss of a friendly relationship
story2 = ["A person who was very close to you, especially in recent times, has to move away unexpectedly. When you parted, you reassured each other you would both keep in close contact. But his/her new home is quite far away. You could see each other only rarely, if at all.", 
          "In the meantime, some weeks have passed. The person hasn’t gotten in touch with you again. Nevertheless, you feel from time to time that you miss him/her.",
          "Finally, it has become clear that your friendship is not the same anymore. Your relationship with other people can’t replace what you have lost. Now and then, you feel disappointed about the relationship you have lost."]

## Aversive: Reproaches from colleagues about professional commitment
story3 = ["You are together with some colleagues. One says that you don't pull your weight when there is difficult work. He claims that you don't think of other colleagues.", 
          "Sometime later, another colleague hints that the problem is not that you don’t think of others but that you lack any real interest in the work.",
          "Finally, you realize what your colleagues were really getting at, and you, for your part, were able to convince them that you sometimes are more cautious at your work than others."]

## Lose&Failure: Loss of a percious object
story4 = ["You cannot find something which is very precious to you. It is associated with the memory of someone who is/was important to you. You have to face up to the possibility that you have lost it.", 
          "Meanwhile, the time has passed. The missing object, which is so important to you did not appear again.",
          "The object seems to definitely be lost. However, you manage to accept this by thinking of other things and memories which are just as precious to you."]

## Lose&Failure: Lose of a project
story5 = ["You have invested a lot of time and effort in your work project. It is really important for you that you finish this project in a few days and that it is done well. Then you realize that important material is not available. It is vital you have it in order to finish the work.", 
          "Shortly before the deadline, you realize that you have not found a way to get the material you need so badly.",
          "In the end, you concentrated more on other aspects of your task and did a successful job of it. You finally finished the project late, but you did your work well."]

## Aversive: critism from acquinted persons
story6 = ["You are with others, friends, and some people you don’t know. You have just done something embarrassing. Somebody has noticed it and brought it to the attention of others. All around, people are laughing at you. You can also hear some impertinent remarks.", 
          "Sometime later, people hadn't yet calmed. They are still making sarcastic remarks about your predicament and are still laughing about it.",
          "People still hadn't calmed down when you left. They were still being cheeky and belittling regarding your mistake."]

## Lose&Failure: Loss of activities/projects 
story7 = ["Your workplace will soon be reorganized. When it happens, you should finish one aspect of the work you have been doing until now. It is work that is very familiar to you and that you have mastered. In its place, you will have to take on other tasks that are as yet undefined and probably not so easy for you. You will miss your previous work very much.", 
          "After a trial period, the new setup becomes definite. Your workplace is to be reorganized. The new tasks you have taken over are rather difficult and don't satisfy you. You're really missing your previous work.",
          "You didn't get on very well with your new work. You miss your previous work very much. Other aspects of your work don’t compensate for this."]

## Aversive: Critisim by a stranger  
story8 = ["Walking down a crowded street, you strike a parked car with your bag. The driver, who was just about to get in, inspects his car. He becomes very angry and blames you for having scratched the car. He becomes abusive towards you and wants you to pay for the damage.", 
          "The driver continues to be very angry. Again he loudly demands that you should pay for the damage.",
          "The driver remained fairly angry. But finally, he seemed to accept that the scratches on his car were hardly likely to have been made by your bag."]

## Aversive: Being charged with a difficult job by the boss 
story9 = ["Previous relations with your boss have been quite complicated. Now your boss gives you a task which you are supposed to work on for the next two days. This job is very inconvenient for you because you have a lot of routine work to do at the moment.", 
          "Your boss tells you that your routine work also has to be done. As you begin to work on the new task, it becomes evident how difficult and time-consuming it really is. It seems that you will only finish it if you ignore your other work, and even then, you may have to work overtime on it.",
          "You didn’t carry out the job in the assigned time. Also, a lot of routine work remained unfinished."]

## Aversive: argument about problems in a relationship
story10 = ["You have just got to know someone you find pretty sympathetic. You have met several times and got on well together. Now you are talking about something which is important to you. The other person does not agree at all and says that your ideas and arguments seem “rather strange” to her.", 
          "Later the person says that she really goes to great lengths to understand you, but she does not manage to agree with you. Obviously, you could talk about this for a long time, she says, but maybe both of you see the world differently.",
          "The disagreements between you remained until the end of the meeting. It was never clear to you what the disagreement was actually about."]

## Lose&Failure: Lose of an interesting side job
story11 = ["Your firm recently advertised a very interesting job. You have the impression that you would be a possible candidate with your professional qualifications. For different reasons, you are hesitating to apply, although a fast decision is needed. Now you hear that one of your colleagues has applied for this job. He is qualified for this job, but you don’t like him.", 
          "Some days later, you hear that your colleague will probably get the job. It seems that he only has to accept the offer.",
          "Your colleague has got the job. Meanwhile, you have concentrated successfully on the other tasks. You have already heard that soon new professional opportunities will be offered which may be of interest to you."]

## Aversive: Reproaches from your partner
story12 = ["You and your partner have been having a difficult time together over the past few weeks. On several occasions, you could have criticized your partner without saying something. Now she/he is saying that she/he finds you “very disagreeable” at present.", 
          "Later, your partner repeats her/his vague reproach. You have only a vague idea as to the background of her/his remark. Your partner seems to be avoiding giving you an explanation.",
          "It remains unclear what your partner means by her/his vague reproaches. You can only guess the reasons. The mutual reproaches are continuing."]

## Lose&Failure: Lose of an important object
story13 = ["You are unable to find an important document that details your professional qualification. You need it urgently.", 
          "After a few days, the missing document still hasn’t appeared. It is highly important you should always have this document at your disposal.",
          "You could not find the certificate in time. You had to show your credentials in another, less satisfactory way."]

## Aversive: Being ignore by your colleagues
story14 = ["Your work colleagues are discussing a professional question in your presence. Although the topic concerns your job and you are familiar with the matter, they don’t attach any importance to your opinion. You have just given your opinion, but your colleagues don’t take notice of what you have said.", 
          "The discussion is continuing. Your colleagues still disregard your opinion.",
          "Finally, you have succeeded in making your opinion known. Some of your colleagues have fully accepted your arguments."]

## Lose&Failure: Lose of an object
story15 = ["You don’t have a good relationship with your landlord. Now he tells you that he is considering canceling your lease. Probably he will need your home for his own use. You are used to the flat, and you like the place and its location.", 
          "Not far from the prescribed cancelation time, your landlord tells you that he is probably going to cancel the lease and use the flat himself.",
          "You have now to move out, and you could not find a similar flat in the vicinity."]

## Aversive: Critisim from a colleague
story16 = ["You have a problem at work. A new colleague that you don’t know well yet is passing and is having a look at your work. Without being unfriendly, he implies that your work could be done in another, more efficient way.", 
          "The new colleague is explaining in a challenging way how the work can be done differently. It is not clear to you what he means. He then adds there may be different ways to solve the problem, but he stresses his “own experiences in the area.",
          "It remains unclear to you if your colleague’s suggestion is valuable or not. His provocative behavior also seems ambiguous to you."]

## Lose&Failure: Failure of a weekend arrangment
story17 = ["You are planning a journey. You intend to visit good friends who live far away. You have not met for a long time, and you have invested a lot of time and energy to ensure the trip came about. Now you receive the message that your visit has possibly to be canceled because of an illness in your friends’ family.", 
          "The day before your departure, you receive a message from your friends that it would probably be better to postpone your visit because of an illness in their family. If you postpone the journey, it will be for an indefinite time.",
          "Finally, you could not visit your friends. You have changed your program, and you have fixed with your friends a new date for a visit."]

## Lose&Failure: Loss of a cooperative work relationship (a colleague leaves)
story18 = ["A work colleague has recently left your firm. His departure surprised you. You worked well with him, and you have received a lot of professional support from him. When he was leaving, you both agreed to remain in contact and continue the professional exchange.", 
          "Meanwhile, the time has passed. After only one spontaneous contact by phone, you have never heard anything from him. Nevertheless, you continue to feel that you are missing his cooperation and support.",
          "The professional exchange is indeed not the same as in former times, but from time to time, you continue to have contact with your colleague, and you meet occasionally. You have also begun to cooperate with other colleagues at the firm, and you receive a lot of professional support from them."]

perrez_stories = [story1, story2, story3, story4, story5, story6, story7, story8,
                  story9, story10, story11, story12, story13, story14, story15,
                  story16, story17, story18]

#Declaring Perrez Questionnaire
There are 6 sets of questions in total: 3 for each time and 2 for each type.  The questionnair is slightly modified.

In [5]:
## Lists of questions (maybe dict is better.)

p1_aversive = []
p2_aversive = []
p3_aversive = []

p1_lossfailure = []
p2_lossfailure = []
p3_lossfailure = []

## Both types share the same measurments for all questions
### We can just concat the answers to the question but doing this makes it easier to exclude answers later
p1_measurement = []
p2_measurement = []
p3_measurement = []

p1_Qnames = []
p2_Qnames = []
p3_Qnames = []

## Emotion Questions
### Emotion questions are always the same across types and times.
### Q 1- 3

eq_nervous = r"""Q. In this situation, I feel:"""
eq_nervous_meas = r"""
  0 very nervous/anxious
  1 fairly nervous/anxious
  2 somewhat nervous/anxious
  3 somewhat calm/composed
  4 fairly calm/composed
  5 very calm/composed
"""

eq_depress = r"""Q. In this situation, I feel:"""
eq_depress_meas = r"""
  0 very depressed/sad
  1 fairly depressed/sad
  2 somewhat depressed/sad
  3 somewhat cheerful/serene
  4 fairly cheerful/serene
  5 very cheerful/serene
"""

eq_angry = r"""Q. In this situation, I feel:"""
eq_angry_meas = r"""
  0 very angry/furious
  1 fairly angry/furious
  2 somewhat angry/furious
  3 somewhat gentle/peaceful
  4 fairly gentle/peaceful
  5 very gentle/peaceful
"""

p1_aversive.extend([eq_nervous,eq_depress,eq_angry])
p2_aversive.extend([eq_nervous,eq_depress,eq_angry])
p3_aversive.extend([eq_nervous,eq_depress,eq_angry])
p1_lossfailure.extend([eq_nervous,eq_depress,eq_angry])
p2_lossfailure.extend([eq_nervous,eq_depress,eq_angry])
p3_lossfailure.extend([eq_nervous,eq_depress,eq_angry])

p1_measurement.extend([eq_nervous_meas,eq_depress_meas,eq_angry_meas])
p2_measurement.extend([eq_nervous_meas,eq_depress_meas,eq_angry_meas])
p3_measurement.extend([eq_nervous_meas,eq_depress_meas,eq_angry_meas])

p1_Qnames.extend(["nervous_calm_1","depressed_cheerful_1","angry_gentle_1"])
p2_Qnames.extend(["nervous_calm_2","depressed_cheerful_2","angry_gentle_2"])
p3_Qnames.extend(["nervous_calm_3","depressed_cheerful_3","angry_gentle_3"])

## Appraisal Questions
### Same across two types. 
### P1 includes change, control, valence, famility
### P2 includes change, control, valence
### P3 includes valence.
### All questions have the same likert scale

appraisal_likert = r"""
  0 very small
  1 fairly small
  2 small
  3 large
  4 fairly large
  5 very large
"""
## Q4 - 7
changeability  = r"""Q. My judgemnts about this situation are as follows: the chances of this situation taking a turn for the better without effort on my part are""" 
controlability = r"""Q. My judgemnts about this situation are as follows: the chances that I can influence this situation for the better are""" 
valence        = r"""Q. My judgemnts about this situation are as follows: the overall amount of stress for me in this situation is""" 
familiarity       = r"""Q. My judgemnts about this situation are as follows: I have experienced a similar situation""" 

p1_aversive.extend([changeability, controlability, valence, familiarity])
p2_aversive.extend([changeability, controlability, valence])
p3_aversive.extend([valence])
p1_lossfailure.extend([changeability, controlability, valence, familiarity])
p2_lossfailure.extend([changeability, controlability, valence])
p3_lossfailure.extend([valence])

p1_measurement.extend([appraisal_likert]*4)
p2_measurement.extend([appraisal_likert]*3)
p3_measurement.extend([appraisal_likert]*1)

p1_Qnames.extend(["changeability_1","controlability_1","valence_1","familiarity_1"])
p2_Qnames.extend(["changeability_2","controlability_2","valence_2"])
p3_Qnames.extend(["valence_3"])

## Coping Goals and Intentions Questions
### Slightly difference between Aversive and Lose&Failure
### Phase 3 doesn't have Coping goals and intentions (since it's already done.)
### Important likery scale

important_scale = r""" 
  0 not important at all
  1 not very important
  2 quite important
  3 very important
"""
## Q8 - 13
allo_problem_aver = r"""Q. In this situation my intention is to actively confront the other(s) and to clear up what is at stake""" 
 
auto_problem_aver = r"""Q. In this situation, my intention is to maintain a friendly atmosphere and to prevent an argument with the other(s)""" 

allo_problem_loss = r"""Q. In this situation, my intention is to maintain the relationship (or objective) which is at stake""" 

auto_problem_loss = r"""Q. In this situation, my intention is to find alternative relationships (or objectives)""" 

emo_ef =  r"""Q. In this situation, my intention is to remain calm and composed""" 

self_esteem_ef = r"""Q. In this situation, my intention is to keep my self-esteem""" 

p1_aversive.extend([allo_problem_aver, auto_problem_aver, emo_ef, self_esteem_ef])
p2_aversive.extend([allo_problem_aver, auto_problem_aver, emo_ef, self_esteem_ef])
p1_lossfailure.extend([allo_problem_loss, auto_problem_loss, emo_ef, self_esteem_ef])
p2_lossfailure.extend([allo_problem_loss, auto_problem_loss, emo_ef, self_esteem_ef])

p1_measurement.extend([important_scale]*4)
p2_measurement.extend([important_scale]*4)

p1_Qnames.extend(["allo_problem_1","auto_problem_1","emotion_focused_ef_1","self_esteem_ef_1"])
p2_Qnames.extend(["allo_problem_2","auto_problem_2","emotion_focused_ef_2","self_esteem_ef_2"])

## Attribution of the outcome 
### Only for the phase 3
### Same for both types

attribute_scale = r""" 
  0 not at all
  1 partially 
  2 mostly 
  3 entirely
"""
### Q29 - 31
self_attribute = r"""Q. I would attribute the outcome of this situation to my own behavior""" 
others_attribute = r"""Q. I would attribute the outcome of this situation to the behavior of the other(s)""" 
circumstances_attribute = r"""Q .I would attribute the outcome of this situation to circumstances"""

p3_aversive.extend([self_attribute, others_attribute, circumstances_attribute])
p3_lossfailure.extend([self_attribute, others_attribute, circumstances_attribute])

p3_measurement.extend([attribute_scale]*3)

p3_Qnames.extend(["self_attribute_3","others_attribute_3","circumstances_attribute_3"])

## Self-directed Actions/Coping 
### Same for both types
### Slightly difference for phase 3. 
### Scale is probability that you would do the action

prob_scale = r"""
  0 not at all (0%)
  1 hardly (25%)
  2 perhaps (50%)
  3 probably (75%)
  4 certainly (100%)
"""
### Q14 - Q19

act_suppressed_info = r"""Q. In this situation, I would fade out, stop paying attention or look for distractions""" 
act_suppressed_info3 =  r"""Q. In this situation, I would distance myself from what happened, stop paying attention or look for distractions""" 

## (Not quite search for info but this is what Perrez used.)
act_search_info = r"""Q. In this situation, I would make clear to myself what is at stake and what I should do""" 
act_search_info3 = r"""Q. In this situation, I would make clear to myself what was at stake and what I could have done better""" 

act_reevaluation = r"""Q. In this situation, I would make clear to myself that this situation is not as straining/important as other problems""" 

act_palliation = r"""Q. In this situation, I would get my emotions under control (be positive, relax, take a drink, a cigarette, etc.)""" 

p1_aversive.extend([act_suppressed_info,act_search_info,act_reevaluation,act_palliation])
p2_aversive.extend([act_suppressed_info,act_search_info,act_reevaluation,act_palliation])
p3_aversive.extend([act_suppressed_info3,act_search_info3,act_reevaluation,act_palliation])
p1_lossfailure.extend([act_suppressed_info,act_search_info,act_reevaluation,act_palliation])
p2_lossfailure.extend([act_suppressed_info,act_search_info,act_reevaluation,act_palliation])
p3_lossfailure.extend([act_suppressed_info3,act_search_info3,act_reevaluation,act_palliation])

p1_measurement.extend([prob_scale]*4)
p2_measurement.extend([prob_scale]*4)
p3_measurement.extend([prob_scale]*4)

p1_Qnames.extend(["act_suppressed_info_1","act_search_info_1","act_reevaluation_1","act_palliation_1"])
p2_Qnames.extend(["act_suppressed_info_2","act_search_info_2","act_reevaluation_2","act_palliation_2"])
p3_Qnames.extend(["act_suppressed_info3_3","act_search_info3_3","act_reevaluation_3","act_palliation_3"])

## Separated out into two batches here so that no batch has more than 5 questions 
act_blame_other = r"""Q. In this situation, I would blame or reproach the other(s) or circumstances by myself"""
act_blame_self = r"""Q. In this situation, I would blame or reproach myself""" 

p1_aversive.extend([act_blame_other,act_blame_self])
p2_aversive.extend([act_blame_other,act_blame_self])
p1_lossfailure.extend([act_blame_other,act_blame_self])
p2_lossfailure.extend([act_blame_other,act_blame_self])

p1_measurement.extend([prob_scale]*2)
p2_measurement.extend([prob_scale]*2)

p1_Qnames.extend(["act_blame_other_1","act_blame_self_1"])
p2_Qnames.extend(["act_blame_other_2","act_blame_self_2"])

## Environment-directed coping
### Two types are different.
### Phase 3 is different but Perrez didn't look at them so we skip them.

### Q23, Q24, Q25
env_passive_aversive = r"""Q. In this situation, I would behave passively or wait for something to happen""" 
env_evasive_aversive = r"""Q. In this situation, I would try to withdraw from the situation (e.g., by avoiding certain things/people or turning away)""" 
env_active_aversive = r"""Q. In this situation, I would try to actively influence the situation (e.g., by questioning, asking for clarification)""" 

### Q26, Q27, Q28
env_passive_loss = r"""Q. In this situation, I would behave passively or wait for something to happen""" 
env_active_prevent_loss = r"""Q. In this situation, I would try to actively prevent a loss or a failure""" 
env_active_reorientate_loss = r"""Q. In this situation, I would try to actively re-orientate my position (e.g., by focusing on other relationships, things, projects)""" 

p1_aversive.extend([env_passive_aversive, env_evasive_aversive, env_active_aversive])
p2_aversive.extend([env_passive_aversive, env_evasive_aversive, env_active_aversive])
p1_lossfailure.extend([env_passive_loss, env_active_prevent_loss, env_active_reorientate_loss])
p2_lossfailure.extend([env_passive_loss, env_active_prevent_loss, env_active_reorientate_loss])

p1_measurement.extend([prob_scale]*3)
p2_measurement.extend([prob_scale]*3)

p1_Qnames.extend(["env_passive_1","env_evasive_1","env_active_1"])
p2_Qnames.extend(["env_passive_2","env_evasive_2","env_active_2"])

#env_p3_active1 = r"""Q20. In this situation, I would say to myself, all that, I did quite well""" + prob_scale
#env_p3_active_reorientate = r"""Q21. In this situation, I would talk to an initimate friend or another person about it""" + prob_scale
#env_p3_active_prevent = r"""Q22. In this situation, I would intend to make it different next time""" + prob_scale

aversive_questions = [p1_aversive, p2_aversive, p3_aversive]
lossfailure_questions = [p1_lossfailure, p2_lossfailure, p3_lossfailure]
measurements = [p1_measurement, p2_measurement, p3_measurement]
question_names = [*p1_Qnames, *p2_Qnames, *p3_Qnames]
p1_batch = [3, 4, 4, 4, 2, 3]
p2_batch = [3, 3, 4, 4, 2, 3]
p3_batch = [3, 1, 3, 4]
batches = [p1_batch, p2_batch, p3_batch]
appraisal_questions = [[3, 4, 5], [3, 4, 5], [3]]


#Genearating Results for Perrez stories
- Perroz's Hypothesis: 
  - 1) Effects of situation and process characteristics on subjective appraisals and on emotional reactions of a stressful episode: 
    - 1.1) According to the objectives of the construction, subjects should perceive **higher controllability and changeability** in the aversive episdoes than in the loss or failure episodes 
    - 1.2) In both types of stressful situations **controllability and changeability should be perceived as lower over the course of the episodes**, from phase 1 to phase 2, due to the stressor remaining unchanged over time.
    - 1.3) Negative valence should be similarly perceived in the two subtests (aver vs loss), but should be in the course of the (unchanged) stressful episdoes. There should be a** strong difference between positive and negative outcome** as described in the hypothetical end of the episode (phase 3)
    - 1.4) Concerning the theme or content of the two classes of stressful episdoes, it is to be supposed that aversive stimuluation **by social agents (as presented in some episodes) produces more aggresive and less depressed emotions** than the looming loss or failure events.
  - 2) Effects of situation and process on situation-directed behaviors:
    - 2.1) **The less controllable** the situations (gross and net control),** the less active influence** on the stressor will be exerted by the subject and,
    - 2.2) **the more passivity** (hesitation and resignation) will occur.
  - 3) Effects on representation-directed coping efforts: **The lower the ambiguity** of the stressful situation then
    - 3.1) **the more limited the serach for information**, and
    - 3.2) **the more the subject will surppress information** about the stressor. 
  - 4) Evaluation-directed coping efforts: **The less controllable** the situation, 
    - 4.1) **The more the instrumental goals (of influence, substituation) are changed from more alloplastic to more autoplastic adaptation**, especially when stressful effects are long-lasting; because social aversive stressors as convceived in the SCPQ episodes probably represent a stronger attack on the self-esteem of the subject, his or her coping intentions should focus more on the goal of (re)equilibration of self-esteem;
    - 4.2) **the more re-evaluation is increased to change the relavance or valence of the actual situation.** 

In [17]:
## Query all Pereze Stories 
wait_time = 3

## After version
instruction_choice = r"""Try to clearly imagine the above situation and then answer the question with the choice only in one line.
"""
instruction_num = r"""Try to clearly imagine the above situation and then answer the question with the choice's number only in one line.
"""
instruction_depressed = r"""I want you to act as a person with depression. Try to clearly imagine the above situation and then answer the question with the choice only in one line.
"""


instructions = [instruction_choice, instruction_num, instruction_depressed]

batch_instruction_choice = r"""Try to clearly imagine the above situation and then answer all the following question(s) with the choice only in one line separated by a comma (e.g., "1, 2, 3").
"""
batch_instruction_num = r"""Try to clearly imagine the above situation and then answer all the following question(s) with the choice's number only in one line separated by a comma (e.g., "1, 2, 3").
"""
batch_instruction_depressed = r"""I want you to act as a person with depression. Try to clearly imagine the above situation and then answer all the following question(s) with the choice's number only in one line separated by a comma (e.g., "1, 2, 3").
"""
batch_instructions = [batch_instruction_choice, batch_instruction_num, batch_instruction_depressed]

## Before version
instruction_bef_choice = r"""Try to clearly imagine the situation below and then answer the question with the choice only in one line.
"""
instruction_bef_num = r"""Try to clearly imagine the situation below and then answer the question with the choice's number only in one line.
"""
instruction_bef_depressed = r"""I want you to act as a person with depression. Try to clearly imagine the situation below and then answer the question with the choice only in one line.
"""

instructions_bef = [instruction_bef_choice, instruction_bef_num, instruction_bef_depressed]

batch_instruction_bef_choice = r"""Try to clearly imagine the situation below and then answer all the following question(s) with the choice only in one line separated by a comma (e.g., "1, 2, 3").
"""
batch_instruction_bef_num = r"""Try to clearly imagine the situation below and then answer all the following question(s) with the choice's number only in one line separated by a comma (e.g., "1, 2, 3").
"""
batch_instruction_bef_depressed = r"""I want you to act as a person with depression. Try to clearly imagine the situation below and then answer all the following question(s) with the choice's number only in one line separated by a comma (e.g., "1, 2, 3").
"""
batch_instructions_bef = [batch_instruction_bef_choice, batch_instruction_bef_num, batch_instruction_bef_depressed]

def getAppraisalAnswers(answer):
  appraisal_answers = ['very small','fairly small','fairly large','very large','small','large'] ##Small and large have to be last.
  for ans in appraisal_answers:
    if answer.find(ans) != -1:
      return ans
  print("NO ANSWER")
  return "no answer."

## Generate the responses from openAI API for one Perrez story
## model = ["text-davinci-003","gpt-3.5-turbo-0301"]
## mode: 1) individual = asking one question at at time.
##       2) batch = asking a batch of the same kind of questions 
## history: append the full history or not. 
def getResponsedPerrezStory(story, type, model, instruction=0, instruct_before = False, history=False, mode="individual", 
                            temperature=0, echo=False):
  out = []
  temp_story = ""
  for i in range(3): 
    temp_story += " " + story[i] ## This will add one empty space before the story. should not be a problem.
    questions = aversive_questions if type == "aversive" else lossfailure_questions
    if mode == "individual":
      for j in range(len(questions[i])):
        prompt = temp_story + "\n" + instructions[instruction]  + questions[i][j] + measurements[i][j] 
        if instruct_before: 
          prompt =  instructions_bef[instruction] + temp_story + "\n"  + questions[i][j] + measurements[i][j]
        response = gptQuery(prompt = prompt, model=model, temperature=temperature)
        
        time.sleep(wait_time) ## Need to wait to not go over api rate limit
        out.extend(response) ## store the response
        if echo:
          print(prompt)
          print(response)
        if history and (j in appraisal_questions[i]): 
          ## Only append appraisal questions 
          temp_story += "\n" + questions[i][j][3:] + " " + getAppraisalAnswers(response[0]) + "." 
      if history: temp_story +="\n" ##Add a new line for the history version since the early parts are followed by appraisal answers
    if mode == "batch":
      ind = 0
      for j in range(len(batches[i])):
        temp_questions = ""
        ## Gather questions
        for k in range(batches[i][j]):
          temp_questions += questions[i][ind] + measurements[i][ind]
          ind += 1
        prompt = temp_story + "\n" + batch_instructions[instruction]  + temp_questions 
        if instruct_before: 
          prompt =  batch_instructions_bef[instruction] + temp_story + "\n"  + temp_questions 
        response = gptQuery(prompt = prompt, model = model, temperature=temperature)
        
        time.sleep(wait_time) ## Need to wait to not go over api rate limit
        out.extend(response)
        if echo:
          print(prompt)
          print(response)

  return out

## Cleaning up answers
def findFirstNumber(txt):
  nums = [int(s) for s in txt if s.isdigit()]
  if len(nums) == 0:
    return
  return nums[0]

## split the response by comma and the gap the first n numbers where n is the number of questions
def postProcessPerrezResponses(responses, batch = []):
  out = []
  for i in range(len(responses)): 
    temp = [findFirstNumber(s) for s in responses[i].split(',')]
    if None in temp: temp.remove(None) ##This would remove comma split without numbers
    while len(temp) < batch[i]: temp.append(None)
    out.extend(temp[0:batch[i]])
  return out


In [8]:
individual_n = [1]*len(question_names) 
batches_n    = [*p1_batch, *p2_batch, *p3_batch]

def genAllPerrezStory(model, mode, instruction=0, instruct_before=False, history = False, temperature=0, n=0, echo = False):
  results = []
  temp_n = individual_n  if mode == "individual" else batches_n
  for i in range(len(perrez_stories)):
    print(i)
    print(f'{perrez_stories[i]}')
    temp = getResponsedPerrezStory(perrez_stories[i], types[i], mode=mode, 
                                 model=model, instruction=instruction, instruct_before = instruct_before,
                                 history=history, temperature = temperature, echo=echo)
    print(temp)
    temp = postProcessPerrezResponses(temp, batch=temp_n)
    print(temp)
    print(len(temp))
    results.append(temp)

  # Saving the output
  df = pd.DataFrame(results, columns = question_names)
  file_name = "Results_" + model + "_" + mode + "_Temp=" + str(temperature) + "_n=" + str(n) + "_instruction=" + str(instruction) + "_instruct_bef=" + str(instruct_before) + "_hist=" + str(history) + ".csv"
  df.to_csv(file_name)

## Don't use anymore
def genOnePerrezStory(story_ind, model, mode, instruction=0, instruct_before=False, temperature=0, n=0, echo=False):
  results = []
  temp_n = individual_n  if mode == "individual" else batches_n
  print(f'{perrez_stories[story_ind]}')
  temp = getResponsedPerrezStory(perrez_stories[story_ind], types[story_ind], mode=mode, 
                                 model=model, instruction=instruction, instruct_before = instruct_before, temperature = temperature, echo=echo)
  print(temp)
  temp = postProcessPerrezResponses(temp, batch=temp_n)
  print(temp)
  print(len(temp))
  results.append(temp)

  # Saving the output
  df = pd.DataFrame(results, columns = question_names)
  file_name = "Results_" + model + "_story=" + str(story_ind) + "_" + mode + "_Temp=" + str(temperature) + "_n=" + str(n) + "_instruction=" + str(instruction) + "_instruct_bef=" + str(instruct_before) + ".csv"
  df.to_csv(file_name)


# Setup 
- 3 x models: 3.5, chat, 4
- 2 x modes of questions: Individual or Batch 
- 4 Instructions
- Total = 24
- Others: 1) Conditioned on early questions and answers within phase 
- Personality/Depressed

Warning: There are 50 questions per story and there are 18 stories so that would be a total of 900 queries. 

### Individual Version

In [ ]:
## Text-davinci-003

# genAllPerrezStory(model=models[0], mode = "individual", instruction = 0, instruct_before=False)

# genAllPerrezStory(model=models[0], mode = "individual", instruction = 1, instruct_before=False)

# genAllPerrezStory(model=models[0], mode = "individual", instruction = 0, instruct_before=True)

# genAllPerrezStory(model=models[0], mode = "individual", instruction = 1, instruct_before=True)

In [ ]:
## GPT3.5-turbo (ChatGPT)

# genAllPerrezStory(model=models[1], mode = "individual", instruction = 0, instruct_before=False)

genAllPerrezStory(model=models[1], mode = "individual", instruction = 1, instruct_before=False)

## Asking to answer with "Choice" can cause Chatgpt to repeat the question or give an answer that just refers to the story.
# genAllPerrezStory(model=models[1], mode = "individual", instruction = 0, instruct_before=True)

# genAllPerrezStory(model=models[1], mode = "individual", instruction = 1, instruct_before=True)

In [ ]:
## GTP4

# genAllPerrezStory(model=models[2], mode = "individual", instruction = 0, instruct_before=False)

# genAllPerrezStory(model=models[2], mode = "individual", instruction = 1, instruct_before=False)

## Got one "I cannot answer this question as I am an AI language model and cannot have personal experiences or judgments."
# genAllPerrezStory(model=models[2], mode = "individual", instruction = 0, instruct_before=True)

# genAllPerrezStory(model=models[2], mode = "individual", instruction = 1, instruct_before=True)

### Batch Version

In [ ]:
## Text-davinci-003

# genAllPerrezStory(model=models[0], mode = "batch", instruction=0)

# genAllPerrezStory(model=models[0], mode = "batch", instruction=1)

# genAllPerrezStory(model=models[0], mode = "batch", instruction=0, instruct_before=True)

# genAllPerrezStory(model=models[0], mode = "batch", instruction=1, instruct_before=True)

In [ ]:
## GPT3.5-turbo (ChatGPT)
### For chatGpt (instruction = 0), a few times (story 2, 12, 15, 18) it didn't answer a few questions (mainly palliation) <- fill with mode
### and also not answer with just numbers for a few blame yourself/other question in p1. 

### For chatGpt (instruction = 1), a few times (story 1, 11, 13, 15, 18), it didn't answer a few questions (again palliation) 
### and also not answer wih just numbers for a few blame self,other, circumstances in p3.

# genAllPerrezStory(model=models[1], mode = "batch", instruction=0)

# genAllPerrezStory(model=models[1], mode = "batch", instruction=1)

# genAllPerrezStory(model=models[1], mode = "batch", instruction=0, instruct_before=True)

# genAllPerrezStory(model=models[1], mode = "batch", instruction=1, instruct_before=True)

In [ ]:
## GPT-4

# genAllPerrezStory(model=models[2], mode = "batch", instruction=0)

# genAllPerrezStory(model=models[2], mode = "batch", instruction=1)

# genAllPerrezStory(model=models[2], mode = "batch", instruction=0, instruct_before=True)

# genAllPerrezStory(model=models[2], mode = "batch", instruction=1, instruct_before=True)

### Conditioned on appraisal answers version
- Only test with instruction with word choices and individual mode 
- Because answering with questions or batch answering doesn't make much sense

In [ ]:
## Only individual

# genAllPerrezStory(model=models[0], mode = "individual", instruction=0, instruct_before=True, history = True)

# genAllPerrezStory(model=models[1], mode = "individual", instruction=0, instruct_before=True, history = True)

# genAllPerrezStory(model=models[2], mode = "individual", instruction=0, instruct_before=True, history = True)

### Act as a person with depression

In [ ]:
## Just Individual 

# genAllPerrezStory(model=models[0], mode = "individual", instruction=2, instruct_before=True)

# genAllPerrezStory(model=models[1], mode = "individual", instruction=2, instruct_before=True)

# genAllPerrezStory(model=models[2], mode = "individual", instruction=2, instruct_before=True)

In [ ]:
## Don't need this anymore
## Combine individual story into one full dataset and save to the file
def mergeInvidual(model, mode, temperature, num, instruction):
    out = []
    for i in range(18):
        file_name = "Results_" + model + "_story=" + str(story_ind) + "_" + mode + "_Temp=" + str(temperature) + "_n=" + str(num) + "_instruction=" + str(instruction) + ".csv"
        out.append(pd.read_csv(file_name))
    file_name = "Results_" + model + "_" + mode + "_Temp=" + str(temperature) + "_n=" + str(num) + "_instruction=" + str(instruction) + ".csv"
    df = pd.concat(out)
    df.to_csv(file_name)
    return(df)


